<div class="alert alert-block alert-info">
    
These blue boxes contain instructions for you to follow, or stuff for you to do
<h2>How to access this Jupyter notebook</h2>

* <b>Step 1</b>: Open a web browser,  and go to [this page](https://jupyter.warwick.ac.uk/module/CH274), 
* <b>Step 2</b>: Enter your SCRTP username and password and press the "Start Server" button.<br>
* <b>Step 3</b>: Wait (it could take a few minutes) until the blue box says "Jupyter notebook server running!". At that point, click on the weblink below said message.<br>
* <b>Step 4</b>: Select the Jupyter Notebook you want to work on. <i>Remember to make a copy of the orginal notebook</i> (which is read-only). To do so, in the toolbar on top of the notebook, select File and then Make a Copy <br>
* <b>Step 5</b>: You're all set! <br>
* <b>Step 6</b>: <font color="red">When you are done, remember to click the "Stop Server" button in the launcher web browser tab.</font> Please do, it's really quite important. <br>
<b> Remember: </b> You can access your copy of the Notebook at any time from any device off and on campus by going through the same steps on e.g. your laptop - all the changes you have made will be saved and synced! <br>

<div/>

# The Hartree Fock method and Psi4

<div class="col-md-10">
<p> In this tutorial we use a the Python scripting language and an interactive Jupyter notebook to run a Hartree-Fock calculation using the Python-based quantum chemistry code Psi4. </p>


<p> You can install this setup on your own Windows/Mac computer following the instructions on the github mainpage.</p>
</div>
<div class="col-md-2">
<img src="http://www.psicode.org/psi4manual/master/_static/psi4square.png" alt="Psi4 code" style="width: 100px;"/>
</div>

You will see lot's of Python statements that import packages or access functions, such as the following:


Here we load important packages, such as a visualization programm and the Psi4 software package and then we do a little arithmetics. We also define a little helper function. 
You can execute the cell below by either pushing the play putton above or by clicking into the cell and pressing "Shift+Enter". You have to do this for all following cells.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import psi4
from ase.build import molecule
from ase.atoms import Atoms
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from psi4_helper import geom_ase_to_psi4

## Part 1: Theory and Method

Hartree-Fock Self-Consistent Field Theory (HF-SCF) with restricted orbitals and closed-shell systems (RHF) solves the so-called Rothaan-Hall equation, which is a generalized eigenvalue matrix equation:

\begin{equation}
\mathbf{FC}=\mathbf{ S C \epsilon}
\end{equation}

The Fock matrix, $\mathbf{F}$, has elements $F_{\mu\nu}$ given as
\begin{equation}
F_{ab} = H_{ab} + \left[ \langle \phi_a|\hat{J}|\phi_b  \rangle - \langle \phi_a|\hat{K}|\phi_b \rangle \right]
\end{equation}
The overlap matrix is defined as
\begin{equation}
S_{ab} = \langle \phi_a|\phi_b \rangle
\end{equation}

The orbital coefficient matrix $\mathbf{C}$ is a $N\times M$ matrix, where $N$ is the number of atomic basis functions ($\phi$) and $M$  is the number of molecular orbitals ($\psi$). This matrix describes the contribution of each basis function $\phi_a$ to the molecular orbital $\psi_i$.

## Part 2: A simple Hartree-Fock calculation

### Step 1: Setting up the molecular coordinates
___________

This is the first step of every calculation. We need to define the molecular coordinates. 
The `ase` package contains a lot of handy tools to define molecules with preset geometries.

In the following, we use the function `molecules` from `ase.build` to build a water molecule.

In [ ]:
#for water, type H2O
#for ammonia, type NH3
#for di-amino-benzene, type BDA,
#for the buckminster-fullerene, type C60
atoms = molecule('H2O')
print(atoms.positions)

Let's visualize the molecule on the screen using `ase.visualize.view`
There are two possible viewers. 'x3d' is the standard one. 'ngl' or nglview has to be installed separately (see handout) but has a lot more features. Try to replace 

`view(atoms, viewer='x3d')` with `view(atoms, viewer='ngl')`

In [ ]:
view(atoms,viewer='x3d') #x3d or ngl

### Step 2: Performing the calculation
___________
To calculate the HF energy, we use the Psi4 package. In the following,

* we set the RAM memory, which we allocate to Psi4, 
* we pass the geometry from our `ase.atoms` object to `psi4.geometry`
* we initiate the HF (also called scf) calculation using a split-valence 3-21G* Pople basis set
* we perform the HF energy calculation using `psi4.energy`

In [ ]:
psi4.core.clean()
#psi4.core.reopen_outfile()
psi4.core.set_output_file('output.dat', False)
psi4.set_memory('1000 MB')

##set geometry
#transform ase geometries to Psi4 input string
geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
#initiate Psi4 molecule object
nh3 = psi4.geometry(geom_input)

# Set computation options
psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type':'direct',
                  'e_convergence': 1e-8})

E, wvf = psi4.energy('scf',return_wfn=True)
print('Total HF energy is {0} Hartree \n '.format(E))


We have successfully calculated the HF energy and wave function. Let's take a closer look at the detailed output statements written in `output.dat`.

In [ ]:
%less output.dat

We can also analyze the results further by looking at the MO energies, the occupations, and the coefficients.

In [ ]:
#Let's look at orbital coefficients, MO energies and occupations
coeffs = wvf.Ca().to_array() #wvfn coefficients
epsilons = wvf.epsilon_a().to_array() #eigenvalues
occs = wvf.occupation_a().to_array() #occupations
for i,(o,e) in enumerate(zip(occs, epsilons)):
    print ('Molecular Orbital {0}, occupation: {1:4.2f}, energy: {2:8.3f}'
           .format(i,o*2,e))
print('\n The wave function matrix has dimensions:', coeffs.shape)
print('\n Wave function coefficients for the lowest energy orbital:')
for i in coeffs[:,0]:
    print('{0:6.2f}'.format(i))    

From the above, we learn that the molecule has 13 AO basis functions, which define 13 Molecular Orbitals. Of these 13 orbitals, 5 are occupied. (These are spatial orbitals, each of which has 2 electrons. That makes 10 electrons in H$_2$O).

The coefficients of the lowest energy orbital tell us that it is only comprised of a single AO, namely the 1s basis function of the oxygen atom. 

The last part will print the molecular wavefunctions and visualize them via rdkit and py3Dmol

In [ ]:
#print wavefunctions as cube files
psi4.cubeprop(wvf)

In [ ]:
#transform molecule into rdkit object to plot it together with MOs
from rdkit import Chem
atoms.write('mol.pdb')
mol_object = Chem.MolFromPDBFile('mol.pdb',removeHs=False)
molblock = Chem.MolToMolBlock(mol_object) 
print(molblock)

In [ ]:
#Visualize molecular orbital number n

#COUNTING STARTS FROM 1
n = 5 #pick the MO to visualize, n=4 is the HOMO
MO_voldata = open("Psi_a_{0}_{0}-A.cube".format(n), "r").read()

import py3Dmol
v = py3Dmol.view()
v.addVolumetricData(MO_voldata, "cube", {'isoval': -0.03, 'color': "red", 'opacity': 0.75}) # negative isovalue
v.addVolumetricData(MO_voldata, "cube", {'isoval': 0.03, 'color': "blue", 'opacity': 0.75}) # positive isovalue
v.addModel(molblock) # plot molecule underneath the orbital
v.setStyle({'stick':{}})
v.zoomTo()
v.show()

## Part 3: Let's play around

<div class="alert alert-block alert-info">

<b> Task: </b><br>

<ul>
<li> Take a look at different Molecular Orbitals of H2O. Print the wave function coefficients and compare to the visualisations of the respective MOs (for example for HOMO-2, HOMO-1, HOMO, LUMO, LUMO+1). </li>
<li> Can you see which atomic orbital combinations constitute the MOs? </li>
<li> Redo the calculation for NH3, BDA and the C60 buckminster fullerene  </li>

</ul>

</div>


In [ ]:
nh3 = molecule('NH3') #pull molecules from database
bda = molecule('BDA') 
bucky = molecule('C60')